In [47]:
# === Superficie apta por umbral de pendiente (con máscara de tierra MapBiomas opcional) ===
# Requisitos: rioxarray, rasterio, geopandas, numpy, xarray

import os, numpy as np, xarray as xr, rioxarray as rxr, geopandas as gpd
import rasterio
from rasterio.features import shapes as rio_shapes
from shapely.geometry import shape
import pandas as pd
import matplotlib.pyplot as plt

In [42]:
# --- Archivos ---
SLOPE_TIF = "/home/aninotna/magister/tesis/justh2_pipeline/data/srtm/slope_deg_aconcagua.tif"
LAND_OK_TIF = "land_ok_2022.tif"   # el que exportaste recién


In [43]:
# 1) Cargar slope
slope = rxr.open_rasterio(SLOPE_TIF, masked=True).squeeze()


In [44]:
# 2) Cargar land_ok
land_ok = rxr.open_rasterio(LAND_OK_TIF, masked=True).squeeze()


In [50]:
# 3) Reproyectar land_ok para que calce con slope
land_ok = land_ok.rio.reproject_match(
    slope,
    resampling=rasterio.enums.Resampling.nearest,  # <- aquí el cambio
    nodata=0
)

In [51]:
# 4) Filtrar pendiente (< umbral)
THRESHOLD_DEG = 10
slope_ok = slope < THRESHOLD_DEG

In [52]:
# 5) Combinar: pendiente baja Y uso de suelo permitido
suitable = xr.where((slope_ok & (land_ok == 1)), 1, 0)


In [53]:

# 6) Exportar a GeoTIFF
suitable = suitable.rio.write_crs(slope.rio.crs)
suitable = suitable.rio.set_spatial_dims(x_dim="x", y_dim="y")
suitable.rio.to_raster("suitable_pv_2022.tif")

print("✅ Exportado suitable_pv_2022.tif")

✅ Exportado suitable_pv_2022.tif
